Installer des versions spécifiques de tensorflow (2.14.0) et keras (2.14.0) pour garantir leur compatibilité.

In [ ]:
!pip install tensorflow==2.14.0
!pip install keras==2.14.0

Afficher la version de tensorflow installée en important la bibliothèque et en imprimant sa version.

In [ ]:
import tensorflow as tf
print(tensorflow.__version__)
print(tf.keras.__version__)

Importer les bibliothèques nécessaires pour manipuler les fichiers, effectuer des calculs numériques, créer et entraîner des réseaux de neurones, gérer des ensembles de données, transformer des images, et afficher une barre de progression.

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from PIL import Image
from tqdm.notebook import tqdm

La fonction load_images permet Charger des images à partir d'un dossier donné, les redimensionne, et les normalise entre -1 et 1, avant de les retourner sous forme de liste.

In [ ]:
# ----------- Fonction de chargement des images -----------
def load_images(data_path, img_shape):
    images = []
    for img_name in os.listdir(data_path):
        img_path = os.path.join(data_path, img_name)
        img = load_img(img_path, target_size=img_shape)
        img = img_to_array(img) / 127.5 - 1  # Normalisation entre -1 et 1
        images.append(img)
    return np.array(images)

Définir un générateur de réseau de neurones pour créer des images à partir d'un vecteur latent, en utilisant des couches entièrement connectées et transposées convolutives, suivi de normalisations et d'activations pour produire une image.

In [ ]:
# ----------- Générateur -----------
def build_generator(latent_dim):
    model = models.Sequential([
        layers.Dense(128 * 32 * 32, activation="relu", input_dim=latent_dim),
        layers.Reshape((32, 32, 128)),
        layers.Conv2DTranspose(64, (4, 4), strides=(2, 2), padding="same"),
        layers.BatchNormalization(),
        layers.LeakyReLU(0.2),
        layers.Conv2DTranspose(3, (4, 4), strides=(2, 2), padding="same", activation="tanh"),
    ])
    return model

Définir un discriminateur, un réseau de neurones convolutif conçu pour différencier les images réelles des images générées, en utilisant des couches convolutives, des normalisations, des activations et une sortie sigmoïde pour produire une probabilité.

In [ ]:
# ----------- Discriminateur -----------
def build_discriminator(img_shape):
    model = models.Sequential([
        layers.Conv2D(64, (4, 4), strides=(2, 2), padding="same", input_shape=img_shape),
        layers.LeakyReLU(0.2),
        layers.Conv2D(128, (4, 4), strides=(2, 2), padding="same"),
        layers.BatchNormalization(),
        layers.LeakyReLU(0.2),
        layers.Flatten(),
        layers.Dense(1, activation="sigmoid"),
    ])
    return model

Initialiser les paramètres du modèle et les hyperparamètres : dimension du vecteur latent (100), taille des images (128x128), chemin des données, taille de batch (64), nombre d'époques (2000), et taux d'apprentissage (0.0002).

In [ ]:
# ----------- Hyperparamètres -----------
latent_dim = 100
img_shape = (128, 128, 3)
data_path = "Dataset/tumor"
batch_size = 64
num_epochs = 2000
lr = 0.0002

Charger les données avec un DataLoader, crée les modèles de générateur et de discriminateur, initialise les optimiseurs avec l'algorithme Adam et définit la fonction de perte (BinaryCrossentropy) pour l'entraînement du GAN.

In [ ]:
# Charger les données
images = load_images(data_path, img_shape[:2])
dataset = tf.data.Dataset.from_tensor_slices(images).batch(batch_size).shuffle(buffer_size=1024)

# Créer les modèles
generator = build_generator(latent_dim)
discriminator = build_discriminator(img_shape)

# Optimiseurs
optimizer_G = Adam(lr, beta_1=0.5)
optimizer_D = Adam(lr, beta_1=0.5)

# Fonction de perte
loss_fn = tf.keras.losses.BinaryCrossentropy()

Entraîner un GAN pendant 2000 époques en optimisant le générateur et le discriminateur. À chaque époque, le générateur crée de fausses images et le discriminateur apprend à distinguer les vraies des fausses. Les pertes des deux modèles sont calculées et les paramètres sont mis à jour. Toutes les 100 époques, des images générées sont sauvegardées.

In [ ]:
# ----------- Entraînement -----------
@tf.function
def train_step(real_images):
    batch_size = tf.shape(real_images)[0]
    valid = tf.ones((batch_size, 1))
    fake = tf.zeros((batch_size, 1))
    
    # Entraîner le générateur
    with tf.GradientTape() as tape:
        z = tf.random.normal((batch_size, latent_dim))
        generated_images = generator(z, training=True)
        g_loss = loss_fn(valid, discriminator(generated_images, training=True))
    gradients_g = tape.gradient(g_loss, generator.trainable_variables)
    optimizer_G.apply_gradients(zip(gradients_g, generator.trainable_variables))
    
    # Entraîner le discriminateur
    with tf.GradientTape() as tape:
        real_loss = loss_fn(valid, discriminator(real_images, training=True))
        fake_loss = loss_fn(fake, discriminator(generated_images, training=True))
        d_loss = 0.5 * (real_loss + fake_loss)
    gradients_d = tape.gradient(d_loss, discriminator.trainable_variables)
    optimizer_D.apply_gradients(zip(gradients_d, discriminator.trainable_variables))
    
    return d_loss, g_loss

Entraîner un GAN pendant 2000 époques en optimisant le générateur et le discriminateur.

In [ ]:
# Boucle d'entraînement
for epoch in tqdm(range(num_epochs), desc="Training GAN"):
# for epoch in range(num_epochs):
    for real_images in dataset:
        d_loss, g_loss = train_step(real_images)
    
    print(f"Epoch {epoch + 1}/{num_epochs} | D Loss: {d_loss:.4f} | G Loss: {g_loss:.4f}")
    
    # Sauvegarde des images toutes les 100 époques
    if epoch % 100 == 0:
        z = tf.random.normal((16, latent_dim))
        generated_images = generator(z, training=False)
        for idx, img in enumerate(generated_images):
            img = (img + 1) / 2.0  # Re-normaliser entre 0 et 1 pour sauvegarde
            tf.keras.preprocessing.image.save_img(f"keras_generated_image_epoch_{epoch}_img_{idx}.png", img)

Génèrer 16 images après l'entraînement à partir de vecteurs aléatoires en utilisant le générateur, et les sauvegarde sous forme de fichiers .png, avec un nom de fichier qui inclut l'index de l'image. Les images sont générées sans calcul de gradients pour économiser de la mémoire.

In [ ]:
# ----------- Générer des images après l'entraînement -----------
z = tf.random.normal((16, latent_dim))
generated_images = generator(z, training=False)
for idx, img in enumerate(generated_images):
    img = (img + 1) / 2.0  # Re-normaliser entre 0 et 1 pour sauvegarde
    tf.keras.preprocessing.image.save_img(f"keras_final_generated_image_{idx}.png", img)

Sauvegarder les poids du modèle du générateur et du discriminateur dans des fichiers .keras afin de pouvoir les recharger et les réutiliser plus tard.

In [ ]:
# Sauvegarder le générateur et le discriminateur après l'entraînement
generator.save("generator_model.keras")
discriminator.save("discriminator_model.keras")

Charger les modèles préalablement sauvegardés en réinitialisant les instances du générateur et du discriminateur, puis charge les poids des modèles sauvegardés.

In [ ]:
from tensorflow.keras.models import load_model

# Charger le générateur et le discriminateur
generator = load_model("generator_model.keras")
discriminator = load_model("discriminator_model.keras")

Génèrer 16 images à partir de vecteurs aléatoires en utilisant le générateur, et les sauvegarde sous forme de fichiers .png, avec un nom de fichier qui inclut l'index de l'image. Les images sont générées sans calcul de gradients pour économiser de la mémoire.

In [ ]:
# Générer des images en utilisant le générateur chargé
z = tf.random.normal((16, latent_dim))  # latent_dim doit correspondre à la dimension utilisée à l'entraînement
generated_images = generator(z)

# Sauvegarder les images générées
for idx, img in enumerate(generated_images):
    img = (img + 1) / 2.0  # Re-normaliser entre 0 et 1 pour sauvegarde
    tf.keras.preprocessing.image.save_img(f"loaded_model_generated_image_{idx}.png", img)
